In [1]:
import pyiast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### AiiDA lab data importation

In [27]:
def conv_to_csv(pressure,loading_absolute_average,mof_gas):
    df = pd.DataFrame({
        'Pressure (bar)': pressure,
        'Loading (mmol/g)': loading_absolute_average
    })

    file_name = f'isotherm_data_{mof_gas}.csv'
    df.to_csv(file_name, index=False)
    return

#import data from the csv file
def import_data(mof_gas):
    file_name = f'isotherm_data_{mof_gas}.csv'
    df_mof = pd.read_csv(file_name)
    return df_mof

#UTSA 80 Aiida lab data
P_CO2 = [0.2, 0.8, 1.4, 2.0, 2.6, 3.2, 3.8, 4.4, 5.0, 5.6, 6.2, 6.8, 7.4, 8.0, 8.6, 9.2, 9.8, 10.2]
q_avg_CO2= [0.31424485381695, 1.1580909367748, 2.038693985348, 2.9664843410745, 3.8041249233618, 4.7398294176147, 5.6300892453135, 6.571769721719, 7.5454467796638, 8.1926294954818, 9.0655394233608, 9.9333829424732, 10.499469115282, 11.176310724031, 11.800907117546, 12.322322186213, 12.73597406036, 13.056237399758]

P_N2 =  [0.2, 0.8, 1.4, 2.0, 2.6, 3.2, 3.8, 4.4, 5.0, 5.6, 6.2, 6.8, 7.4, 8.0, 8.6, 9.2, 9.8, 10.2]
q_avg_N2 = [0.06563379544785, 0.26439683199255, 0.45816146525745, 0.62400984619095, 0.8023627360098, 0.97143287688045, 1.1356066227015, 1.3174702957764, 1.4491204175354, 1.6241835419303, 1.7920465854102, 1.9268249595619, 2.1054243692705, 2.239310171408, 2.337518595033, 2.5546176109562, 2.6749108164195, 2.7594671385695]

mof_CO2 = "UTSA_80_CO2"
mof_N2 = "UTSA_80_N2"

#fichier_CO2 = conv_to_csv(P_CO2,q_avg_CO2,mof_CO2)
#fichier_N2 = conv_to_csv(P_N2,q_avg_N2,mof_N2)

#### Loading q values in pure conditions

In [3]:
def get_pure_q(df_mof):
    pure_isotherm = pyiast.InterpolatorIsotherm(df_mof,
                                        loading_key="Loading (mmol/g)",
                                        pressure_key="Pressure (bar)")
    pure_isotherm = pyiast.InterpolatorIsotherm(df_mof,
                                        loading_key="Loading (mmol/g)",
                                        pressure_key="Pressure (bar)",
                                        fill_value=df_mof['Loading (mmol/g)'].max())
    return pure_isotherm

#### Loading q values in binary mixture

In [4]:
def get_mixt_q(N2_isotherm, CO2_isotherm, pressure):
    p_part = pressure*np.array([0.85,0.15])
    q_mix = pyiast.iast(p_part, [N2_isotherm, CO2_isotherm], verboseflag=False)
    return q_mix

#### selectivity and binary mixture loading calculation

In [19]:
def get_selectivity(mof_N2,mof_CO2, S_pressure):
    name_N2 = f"{mof_N2}"
    name_CO2 = f"{mof_CO2}"

    df_N2 = import_data(name_N2)
    df_CO2 = import_data(name_CO2)
    
    N2_isotherm = get_pure_q(df_N2)
    CO2_isotherm = get_pure_q(df_CO2)
    
    q_mix = get_mixt_q(N2_isotherm, CO2_isotherm, S_pressure)
    Selectivity_mof = (0.85/0.15)*(q_mix[1]/q_mix[0])
    return Selectivity_mof

#### values computation and display

In [29]:
S_pressure = 1 #bar
MOF = ["UTSA_20","UTSA_80","Mg_74","NOTT_300"]
S = []

for i in range(len(MOF)):
    mof_N2 = f'{MOF[i]}_N2'
    mof_CO2 = f'{MOF[i]}_CO2'
    S_mof = get_selectivity(mof_N2,mof_CO2, S_pressure)
    S.append(round(float(S_mof),2))

#print the results
BOLD = "\033[1m"
RESET = "\033[0m"

column1 = ["MOF"] + MOF
print(f"{BOLD}{column1[0]:<15} {'Selectivity':<15}{RESET}")

for item1, item2 in zip(column1[1:], S):
    print(f"{item1:<15} {round(item2, 1):<15}")

MOF             Selectivity    
UTSA_20         8.1            
UTSA_80         4.7            
Mg_74           4.6            
NOTT_300        16.5           
